In [1]:
from pyspark.sql import SQLContext

In [2]:
sqlsc = SQLContext(sc)

In [3]:
df = sqlsc.read.format("jdbc") \
  .option("url", "jdbc:postgresql://localhost/cloudera?user=cloudera") \
  .option("dbtable", "gameclicks") \
  .load()

In [5]:
df.printSchema()

root
 |-- timestamp: timestamp (nullable = false)
 |-- clickid: integer (nullable = false)
 |-- userid: integer (nullable = false)
 |-- usersessionid: integer (nullable = false)
 |-- ishit: integer (nullable = false)
 |-- teamid: integer (nullable = false)
 |-- teamlevel: integer (nullable = false)



In [7]:
df.count()

755806

In [8]:
df.show()

+--------------------+-------+------+-------------+-----+------+---------+
|           timestamp|clickid|userid|usersessionid|ishit|teamid|teamlevel|
+--------------------+-------+------+-------------+-----+------+---------+
|2016-06-09 13:32:...| 438813|   390|        20554|    1|    11|        6|
|2016-06-09 13:32:...| 439001|   446|        20770|    1|    57|        6|
|2016-06-09 13:33:...| 439243|   748|        24389|    0|   103|        6|
|2016-06-09 13:33:...| 439400|  1634|        24100|    1|   123|        5|
|2016-06-09 13:33:...| 439124|  1214|        20929|    0|    82|        6|
|2016-06-09 13:33:...| 439250|  2174|        21283|    1|   104|        6|
|2016-06-09 13:33:...| 438871|   662|        20659|    0|    28|        6|
|2016-06-09 13:33:...| 439630|  2056|        23338|    0|   152|        3|
|2016-06-09 13:33:...| 439052|   109|        20841|    0|    69|        6|
|2016-06-09 13:33:...| 438943|   366|        20708|    0|    39|        6|
|2016-06-09 13:33:...| 43

In [9]:
df.select('userid', 'teamlevel').show()

+------+---------+
|userid|teamlevel|
+------+---------+
|   390|        6|
|   446|        6|
|   748|        6|
|  1634|        5|
|  1214|        6|
|  2174|        6|
|   662|        6|
|  2056|        3|
|   109|        6|
|   366|        6|
|   590|        6|
|   178|        6|
|  2263|        6|
|  1765|        6|
|   579|        3|
|  1815|        6|
|  1355|        5|
|  1771|        6|
|   290|        3|
|  1125|        3|
+------+---------+
only showing top 20 rows



In [10]:
df.filter(df['teamlevel']>1).select('userid', 'teamlevel').show()

+------+---------+
|userid|teamlevel|
+------+---------+
|   390|        6|
|   446|        6|
|   748|        6|
|  1634|        5|
|  1214|        6|
|  2174|        6|
|   662|        6|
|  2056|        3|
|   109|        6|
|   366|        6|
|   590|        6|
|   178|        6|
|  2263|        6|
|  1765|        6|
|   579|        3|
|  1815|        6|
|  1355|        5|
|  1771|        6|
|   290|        3|
|  1125|        3|
+------+---------+
only showing top 20 rows



In [12]:
df.groupBy('ishit').count().show()

+-----+------+
|ishit| count|
+-----+------+
|    0|672423|
|    1| 83383|
+-----+------+



In [13]:
from pyspark.sql.functions import *

In [14]:
df.select(mean('ishit'), sum('ishit')).show()

+------------------+----------+
|        avg(ishit)|sum(ishit)|
+------------------+----------+
|0.1103232840173272|     83383|
+------------------+----------+



In [15]:
df2 = sqlsc.read.format("jdbc") \
  .option("url", "jdbc:postgresql://localhost/cloudera?user=cloudera") \
  .option("dbtable", "adclicks") \
  .load()

In [16]:
merge = df.join(df2, 'userid')

In [17]:
merge.printSchema()

root
 |-- userid: integer (nullable = false)
 |-- timestamp: timestamp (nullable = false)
 |-- clickid: integer (nullable = false)
 |-- usersessionid: integer (nullable = false)
 |-- ishit: integer (nullable = false)
 |-- teamid: integer (nullable = false)
 |-- teamlevel: integer (nullable = false)
 |-- timestamp: timestamp (nullable = false)
 |-- txid: integer (nullable = false)
 |-- usersessionid: integer (nullable = false)
 |-- teamid: integer (nullable = false)
 |-- adid: integer (nullable = false)
 |-- adcategory: string (nullable = false)



In [18]:
merge.show()

+------+--------------------+-------+-------------+-----+------+---------+--------------------+-----+-------------+------+----+-----------+
|userid|           timestamp|clickid|usersessionid|ishit|teamid|teamlevel|           timestamp| txid|usersessionid|teamid|adid| adcategory|
+------+--------------------+-------+-------------+-----+------+---------+--------------------+-----+-------------+------+----+-----------+
|   231|2016-06-09 14:06:...| 440305|        23626|    1|   142|        4|2016-06-08 01:40:...|23669|        23626|   142|  27|      games|
|   231|2016-06-09 14:06:...| 440305|        23626|    1|   142|        4|2016-06-08 09:24:...|24122|        23626|   142|   4|      games|
|   231|2016-06-09 14:06:...| 440305|        23626|    1|   142|        4|2016-06-08 17:21:...|24659|        23626|   142|  22|  computers|
|   231|2016-06-09 14:06:...| 440305|        23626|    1|   142|        4|2016-06-08 23:34:...|25076|        23626|   142|  21|     movies|
|   231|2016-06-09 1